In [16]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import warnings
import re
import joblib
import string
import os
import time
import json

In [17]:
startTime = time.time()
lastTime = startTime
times = dict()

In [18]:
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [19]:
DATA_DIR = './data/'
STATIC_DIR = '../../static/data/'

# Functions and Classes

In [20]:
class Profession:
    __all_data = None
    __name = None
    
    def __init__(self, profession):
        self.__all_data = list()
        self.__name = profession
        
    def add(self, itemID, itemName, reagents, crafterName, tag, difficulty, multicraft, quantity, skill,
            rarity, hasReagentQualities, hasEmbellishmentSlot, hasMissiveSlot, hasSafetyComponent, hasCrestSlot):

            self.__all_data.append([self.__name, crafterName, itemID, itemName, None, reagents, tag, rarity, 
                                    difficulty, skill, quantity, multicraft, hasReagentQualities, 
                                    hasEmbellishmentSlot, hasMissiveSlot, hasSafetyComponent, hasCrestSlot])
            
    def get_table(self):
        columns = ['profession', 'character', 'itemID', 'item', 'icon', 'reagents', 'tag', 'rarity', 
                   'difficulty', 'skill1', 'baseQuantity', 'multicraftPercent', 'hasReagentQualities', 
                   'hasEmbellishmentSlot', 'hasMissiveSlot', 'hasSafetyComponent', 'hasCrestSlot']
        dtypes = ['string', 'string', 'int32', 'string', 'string', 'object', 'string', 'string', float, float, 
                  'string', float, bool, bool, bool, bool, bool, bool]
        df = pd.DataFrame(columns=columns, data=self.__all_data)
        return df.astype(dict(zip(columns, dtypes)))
        return df
        
    def set_table(self, df):
        self.__all_data = df.to_numpy()

In [21]:
def getReagents(professionName, itemID, itemName):
    with open(DATA_DIR+professionName.lower()+'_recipes.json') as file:
        recipes = json.load(file)
        
    for recipe in recipes:
        if recipe.get('itemID')==str(itemID) and recipe.get('itemName')==itemName:
            return recipe.get('reagents')
    else:
        raise KeyError(f'Item ID "{itemID}" with item name "{itemName}" not found')

In [34]:
def getStats(professionName, characterName, itemID, itemName, 
             stats=['skill', 'multicraft', 'resourcefulness', 'ingenuity']):
    assert type(stats)==list, 'stats must be a list of strings'
    for stat in stats:
        assert stat.lower() in ['skill', 'multicraft', 'resourcefulness', 'ingenuity'], f'{stat} is not valid'
    stats = [stat.lower() for stat in stats]
    
    #character profession knowledge points
    knowledge = pd.read_csv(DATA_DIR+professionName+'_knowledge.csv')
    assert characterName in knowledge.columns, f'{characterName} does not know {professionName} [Err:knowledge]'
    knowledge = knowledge.loc[:, ['node', characterName]]
    
    #skill, resourcefulness, ingenuity, and multicraft stats from each node
    with open(DATA_DIR+professionName+'_nodes.json') as file:
        nodes = json.load(file)   
        
    #which nodes affect which recipes
    with open(DATA_DIR+professionName+'_specializations.json') as file:
        specializations = json.load(file)
        for spec in specializations:
            if spec.get('itemID')==str(itemID) and spec.get('itemName')==itemName:
                specializations = spec.get('nodes')
                break
        else:
            raise KeyError(f'Item ID "{itemID}" with item name "{itemName}" not found')
        
    stats = dict(zip(stats, np.zeros(len(stats))))
    for stat in stats.keys():
        statValues = nodes.get(stat)
        for spec in specializations:
            try:
                knowledgePoints = int(knowledge.loc[knowledge['node']==spec, characterName])
            except:
                display(knowledge)
                display(spec)
                display(characterName)
                display(knowledge.loc[knowledge['node']==spec, characterName])
                
            #node not unlocked yet, so don't count it
            if knowledgePoints == -1:
                continue
                
            #apply stat gained per point in spec node
            stats[stat] += int(statValues.get(spec).get('scaling'))*knowledgePoints
            
            #apply stat gained from achieved breakpoints in spec node
            for breakpoint in np.arange(start=0, stop=knowledgePoints+1, step=5):
                stats[stat] += int(statValues.get(spec).get(str(breakpoint)))
        
    
    return stats

In [38]:
def getProfession(professionName, characterName):
    professionName = professionName.lower()
    
    validProfessions = ['alchemy', 'blacksmithing', 'cooking', 'enchanting', 'engineering', 'inscription', 
                        'jewelcrafting', 'leatherworking', 'tailoring']
    assert professionName.lower() in validProfessions, f'{professionName} is not a valid profession'
    
    #base values from profession level and equipment
    baseValues = pd.read_csv(DATA_DIR+'base_stats.csv')
    baseValues = baseValues.loc[(baseValues['profession']==professionName)&(baseValues['character']==characterName),
                                baseValues.columns]
    assert len(baseValues)==1, 'Issue with base values table'
    baseValues = baseValues.to_dict(orient='records')[0]
    
    #attributes for each recipe (tag, difficulty, base stats, rarity, embellishments, etc)
    items = pd.read_csv(DATA_DIR+professionName+'_items.csv')
    
    #which recipes each character knows
    learned = pd.read_csv(DATA_DIR+professionName+'_learned.csv')
    assert characterName in learned.columns, f'{characterName} does not know {professionName} [Err:learned]'
    learned = learned.loc[:, ['itemID', 'itemName', characterName]]
    
    profession = Profession(professionName)
    
    #set crafter to None if not learned by character, otherwise set to character name
    table = pd.merge(left=items, right=learned, on=['itemID', 'itemName'], how='outer', suffixes=[None, None])
    table['crafter'] = None
    try:
        table.loc[table[characterName], 'crafter'] = characterName
    except:
        display(characterName)
        display(table)
        
    for index, row in table.iterrows():   
        itemID = row['itemID']
        itemName = row['itemName']
        
        #adjust values to allow for np.nan since you can't do float('np.nan')
        try:
            skill = float(row['skill'])
        except:
            skill = np.nan
        
        try:
            multicraft = float(row['multicraft'])
        except:
            multicraft = np.nan
            
        try:
            difficulty = float(row['difficulty'])
        except:
            difficulty = np.nan
            
        skill += (baseValues.get('level') + baseValues.get('skill'))
        multicraft += baseValues.get('multicraft')
        multicraft = np.round(multicraft/33*100, 1) #convert to a percent stat rather than stat value
        
        if professionName != 'cooking':
            stats = getStats(professionName, characterName, itemID, itemName)
        else:
            stats = dict(zip(['skill', 'multicraft', 'resourcefulness', 'ingenuity'], [0,0,0,0]))
            
        profession.add(itemID = itemID,
                       itemName = itemName,
                       reagents = getReagents(professionName, itemID, itemName),
                       crafterName = characterName,
                       tag = row['tag'],
                       difficulty = difficulty,
                       multicraft = multicraft + stats['multicraft'],
                       quantity = row['quantity'],
                       skill = skill + stats['skill'],
                       rarity = row['rarity'],
                       hasReagentQualities = row['hasReagentQualities'],
                       hasEmbellishmentSlot = row['hasEmbellishmentSlot'],
                       hasMissiveSlot = row['hasMissiveSlot'],
                       hasSafetyComponent = row['hasSafetyComponent'],
                       hasCrestSlot = row['hasCrestSlot']
                      )
        
    return profession

In [24]:
def scrapeIcon(itemID):
    url = f'https://www.wowhead.com/item={itemID}'
    soup = BeautifulSoup(requests.get(url).text) 
    
    #string1 finds strings preceded by:   "{itemID}:{" 
    #and are also followed by:    ,"screenshot"
    #the strings cannot include the symbol:   }
    string1 = re.search(r'(?<="'+f'{itemID}'+r'":{)[^}]+(?=,"screenshot")', str(soup)).group()
    
    #string2 searches string1 for strings preceded by:      "icon":"
    #and are also followed by:      ")
    #that only contain a-z, A-Z, 0-9, _, and -
    string2 = re.search(r'(?<="icon":")[\w-]+(?=")', string1).group()
    site = 'https://wow.zamimg.com/images/wow/icons/large/'+string2+'.jpg'
    
    status_code = requests.get(site).status_code
    
    if status_code==200:
        return {itemID: site}
    else:
        return {itemID: None}

In [25]:
def check_id(old_id):
    url = f'https://www.wowhead.com/item={old_id}?xml'
    html = requests.get(url).text
    soup = BeautifulSoup(html, features='xml')
    name = soup.find('name').text
    text = soup.find('htmlTooltip').text
    
    #check for quality tier information
    if text.find('quality-tier1') >= 0:
        new_id = old_id+2
        new_id_lower = old_id-2
    elif text.find('quality-tier2') >= 0:
        new_id = old_id+1
        new_id_lower = old_id-1
    else: #either its tier3 or it doesn't have tiers, in which use the old_id
        return {old_id: old_id}
    
    
    #wasn't a tier 3 item, so check the calculated id for if the name matches and is tier 3
    #return the new id if it is the same name and tier 3, else return -1 for manual checking
    try:
        url = f'https://www.wowhead.com/item={new_id}?xml'
        html = requests.get(url).text
        soup = BeautifulSoup(html, features='xml')
        if soup.find('name').text == name and soup.find('htmlTooltip').text.find('quality-tier3') >= 0:
            return {old_id: new_id}
    except:
        pass
    
    try:
        url = f'https://www.wowhead.com/item={new_id_lower}?xml'
        html = requests.get(url).text
        soup = BeautifulSoup(html, features='xml')
        if soup.find('name').text == name and soup.find('htmlTooltip').text.find('quality-tier3') >= 0:
            return {old_id: new_id_lower}
    except:
        return {old_id: -1}
        
    return {old_id: -1}

In [26]:
def outcomeQuality(skill, difficulty, tag):
    if tag.lower()[:4]=='gear':
        arr = np.array([1, 0.2*difficulty, 0.5*difficulty, 0.8*difficulty, difficulty])
    else:
        arr = np.array([1, 0.5*difficulty, difficulty])
        
    return (skill >= arr).sum()

In [27]:
def updateReagents(reagents: dict, replacementIDs: dict):
    return {replacementIDs.get(reagent, reagent):count for reagent,count in reagents.items()}

In [28]:
curTime = time.time()
times['functions'] = curTime - lastTime
lastTime = curTime

# Initial DataFrames

In [29]:
items_columns = ['itemID', 'item', 'icon', 'tag', 'rarity']
items_dtypes = ['int32', 'string', 'string', 'string', 'string']
items = pd.DataFrame(columns=items_columns)

professions_columns = ['profession', 'itemID', 'reagents', 'hasReagentQualities', 'hasEmbellishmentSlot',
                       'hasMissiveSlot', 'hasSafetyComponent', 'hasCrestSlot']
professions_dtypes = ['string', 'int32', dict, bool, bool, bool, bool, bool]
professions = pd.DataFrame(columns=professions_columns)

crafting_columns = ['itemID', 'difficulty', 'character', 'skill1', 'base_quantity', 'multicraft_percent']
crafting_dtypes = ['int32', 'int16', 'string', float, 'string', float]
crafting = pd.DataFrame(columns=crafting_columns)

In [46]:
alchemy = getProfession('Alchemy', 'Trillithia')
alchemy2 = getProfession('Alchemy', 'Sillik')
blacksmithing = getProfession('Blacksmithing', 'Zarastannil')
blacksmithing2 = getProfession('Blacksmithing', 'Nystelil')
cooking = getProfession('Cooking', 'Trillithia')
enchanting = getProfession('Enchanting', 'Linidel')
enchanting2 = getProfession('Enchanting', 'Mellasona')
engineering = getProfession('Engineering', 'Trillithia')
inscription = getProfession('Inscription', 'Mellasona')
inscription2 = getProfession('Inscription', 'Lindinil')
jewelcrafting = getProfession('Jewelcrafting', 'Nystelil')
jewelcrafting2 = getProfession('Jewelcrafting', 'Zarastannil')
leatherworking = getProfession('Leatherworking', 'Braevele')
leatherworking2 = getProfession('Leatherworking', 'Lindinil')
tailoring = getProfession('Tailoring', 'Linidel')
tailoring2 = getProfession('Tailoring', 'Braevele')

# DataFrame Merging

In [ ]:
#concatenate tables
#sort by itemID and skill (descending) so items are paired with higher skill on top
#keep the first entry for each itemID (i.e., the highest skill entry)
#break same skill tie by sorting by name such that primary crafter is at the top
sortCols = ['itemID', 'skill1', 'character']
sortVals = [True, False]
nameAscending = {'alchemy': False,
                 'blacksmithing': False,
                 'enchanting': True,
                 'inscription': False,
                 'jewelcrafting': True,
                 'leatherworking': True,
                 'tailoring': False}

all_data = pd.DataFrame()
for df1, df2, name in [(alchemy, alchemy2, 'alchemy'), (blacksmithing, blacksmithing2, 'blacksmithing'),
                       (enchanting, enchanting2, 'enchanting'), (inscription, inscription2, 'inscription'),
                       (jewelcrafting, jewelcrafting2, 'jewelcrafting'), 
                       (leatherworking, leatherworking2, 'leatherworking'), (tailoring, tailoring2, 'tailoring')]:
    table1 = df1.get_table()
    table2 = df2.get_table()
    
    #recipes known by at least one
    known = pd.concat((table1[table1['character']!='None'], table2[table2['character']!='None']), ignore_index=True)
    
    #recipes not known by both
    unknown = pd.concat((table1[table1['character']=='None'], table2[table2['character']=='None']), ignore_index=True)
    
    #only recipes with known crafters are included, so sort by highest skill, then chosen character order
    #and only take the first (i.e. the character we want to be displayed)
    known = known.sort_values(by=['itemID', 'skill1', 'character'], ascending=[True, False]+[nameAscending[name]])
    known = known.groupby('itemID', as_index=False).first()
    
    #all recipes have crafter as None, so take only the recipe with highest skill if there are duplicates
    unknown = unknown.sort_values(by=['itemID', 'skill1'], ascending=[True, False])
    unknown = unknown.groupby('itemID', as_index=False).first()
    
    #*should* (I think) keep known recipes then unknown recipes in that order. So grouping by itemID should put
    #known ones before unknown and hence only take the prechosen known recipe, unless it was known by no character
    #in which it will only appear once anyway with character=='None', which .first() will collect
    df = pd.concat((known, unknown), ignore_index=True)    
    
    df = df.groupby('itemID', as_index=False).first()
    all_data = pd.concat((all_data, df), ignore_index=True)

In [ ]:
all_data = pd.concat((all_data, cooking.get_table()), ignore_index=True)
all_data = pd.concat((all_data, engineering.get_table()), ignore_index=True)
all_data = all_data.reset_index(drop=True)

# Manual Adjustments

In [ ]:
fixes = {'"Magically ""Infinite"" Messenger"': 'Magically "Infinite" Messenger'}
all_data['item'] = all_data['item'].apply(lambda x: fixes.get(x, x))

In [ ]:
curTime = time.time()
times['base frames'] = curTime - lastTime
lastTime = curTime

# Items DataFrame

In [ ]:
#single dataframe of all items, including those listed in reagents
columns = ['itemID', 'item', 'icon']
items = all_data.loc[:, ['itemID', 'item', 'icon']]
items = items.drop_duplicates()
        
for index, row in tqdm(all_data.iterrows(), total=len(all_data)):
    for reagent in row['reagents'].keys():
        if reagent not in items.loc[:, 'itemID'].to_numpy():
            url = f'https://www.wowhead.com/item={reagent}?xml'
            html = requests.get(url).text
            soup = BeautifulSoup(html, features='xml')
            name = soup.find('name').text
            df = pd.DataFrame(columns=columns, data=[[reagent, name, None]])
            items = pd.concat((items, df))

In [ ]:
curTime = time.time()
times['items lookup'] = curTime - lastTime
lastTime = curTime

# Item Icons

In [ ]:
method = "LOAD"
icon_file = STATIC_DIR+'icons.pkl'

if not os.path.isfile(icon_file) or method == "UPDATE":
    num_cores = joblib.cpu_count()
    all_jobs = [joblib.delayed(scrapeIcon)(itemID) for itemID in items['itemID'].values]
    results = joblib.Parallel(n_jobs=num_cores, verbose=10)(all_jobs)
    icon_links = {int(k):v for d in results for k,v in d.items()}
    icons_df = pd.DataFrame()
    icons_df['itemID'] = icon_links.keys()
    icons_df['link'] = icon_links.values()
    icons_df.to_pickle(icon_file)
elif os.path.isfile(icon_file) and method == "LOAD":
    icon_links = pd.read_pickle(icon_file)
    icon_links = dict(zip(icon_links['itemID'].values, icon_links['link']))

In [ ]:
#update icons in dataframes
all_data['icon'] = all_data['itemID'].map(icon_links)
items['icon'] = items['itemID'].map(icon_links)

In [ ]:
#ensure all items have icons
df = all_data.loc[all_data['icon'].isna(), ['itemID', 'icon']]
assert(len(df)==0)

In [ ]:
curTime = time.time()
times['icons'] = curTime - lastTime
lastTime = curTime

# Update ItemIDs to be Rank 3

In [ ]:
method = "LOAD"
itemIDfile = STATIC_DIR+'itemIDUpdates.pkl'

if not os.path.isfile(itemIDfile) or method == 'UPDATE':
    old_ids = items['itemID'].astype(int).to_numpy()
    all_jobs = [joblib.delayed(check_id)(old_id) for old_id in old_ids]
    results = joblib.Parallel(n_jobs=num_cores, verbose=10)(all_jobs)
    new_ids = {int(k):int(v) for d in results for k,v in d.items()}
    new_ids_df = pd.DataFrame()
    new_ids_df['oldID'] = new_ids.keys()
    new_ids_df['newID'] = new_ids.values()
    new_ids_df.to_pickle(itemIDfile)
elif os.path.isfile(itemIDfile) and method == 'LOAD':
    new_ids = pd.read_pickle(itemIDfile)
    new_ids = dict(zip(new_ids['oldID'].values, new_ids['newID'].values))

In [ ]:
#manual ID fixes
for old_id in new_ids.keys():
    if old_id in range(224300, 224324): #gleeful glamours
        new_ids[old_id] = old_id+48
    elif old_id == 219952: #refulgent crystal
        new_ids[old_id] = 219955
    elif old_id == 212670: #thunderous hide
        new_ids[old_id] = 212673

In [ ]:
#ensure all items with a rank 3 are listed at rank 3
assert -1 not in new_ids.values()

In [ ]:
#update itemIDs in both dataframes
all_data['itemID'] = all_data['itemID'].map(new_ids)
items['itemID'] = items['itemID'].map(new_ids)

In [ ]:
#update all item ids that are not at rank 3 with the found rank 3 ids
all_data['reagents'] = all_data['reagents'].apply(updateReagents, args=(new_ids,))

In [ ]:
curTime = time.time()
times['item qualities'] = curTime - lastTime
lastTime = curTime

# Add Difficulties

In [ ]:
#all mats rank 2
all_data['skill2'] = all_data['skill1']+all_data['difficulty']*0.2

#all mats rank 3
all_data['skill3'] = all_data['skill1']+all_data['difficulty']*0.4

extraDiff = {'safetyComponent':10,
             'missive':5,
             'embellishment':5,
             'weatheredCrest':100,
             'runedCrest':10,
             'gildedCrest':20,
             'combatant':0,
             'aspirant':50,
             'gladiator':150}

#safetycomponent+embellishment not currently possible
all_data['difficulty_safetycomponent'] = all_data['difficulty'] + extraDiff['safetyComponent']
all_data['difficulty_missive'] = all_data['difficulty'] + extraDiff['missive']
all_data['difficulty_embellishment'] = all_data['difficulty'] + extraDiff['embellishment']
all_data['difficulty_safetycomponent_missive'] = all_data['difficulty']
all_data['difficulty_missive_embellishment'] = all_data['difficulty'] + extraDiff['missive'] + extraDiff['embellishment']
all_data['difficulty_weathered'] = all_data['difficulty'] + extraDiff['weatheredCrest']
all_data['difficulty_weathered_safetycomponent'] = all_data['difficulty'] + extraDiff['weatheredCrest'] + extraDiff['safetyComponent']
all_data['difficulty_weathered_missive'] = all_data['difficulty'] + extraDiff['weatheredCrest'] + extraDiff['missive']
all_data['difficulty_weathered_embellishment'] = all_data['difficulty'] + extraDiff['weatheredCrest'] + extraDiff['embellishment']
all_data['difficulty_weathered_safetycomponent_missive'] = all_data['difficulty'] + extraDiff['weatheredCrest'] + extraDiff['safetyComponent'] + extraDiff['missive']
all_data['difficulty_weathered_missive_embellishment'] = all_data['difficulty'] + extraDiff['weatheredCrest'] + extraDiff['missive'] + extraDiff['embellishment']
all_data['difficulty_runed'] = all_data['difficulty'] + extraDiff['runedCrest']
all_data['difficulty_runed_safetycomponent'] = all_data['difficulty'] + extraDiff['runedCrest'] + extraDiff['safetyComponent']
all_data['difficulty_runed_missive'] = all_data['difficulty'] + extraDiff['runedCrest'] + extraDiff['missive']
all_data['difficulty_runed_embellishment'] = all_data['difficulty'] + extraDiff['runedCrest'] + extraDiff['embellishment']
all_data['difficulty_runed_safetycomponent_missive'] = all_data['difficulty'] + extraDiff['runedCrest'] + extraDiff['safetyComponent'] + extraDiff['missive']
all_data['difficulty_runed_missive_embellishment'] = all_data['difficulty'] + extraDiff['runedCrest'] + extraDiff['missive'] + extraDiff['embellishment']
all_data['difficulty_gilded'] = all_data['difficulty'] + extraDiff['gildedCrest']
all_data['difficulty_gilded_safetycomponent'] = all_data['difficulty'] + extraDiff['gildedCrest'] + extraDiff['safetyComponent']
all_data['difficulty_gilded_missive'] = all_data['difficulty'] + extraDiff['gildedCrest'] + extraDiff['missive']
all_data['difficulty_gilded_embellishment'] = all_data['difficulty'] + extraDiff['gildedCrest'] + extraDiff['embellishment']
all_data['difficulty_gilded_safetycomponent_missive'] = all_data['difficulty'] + extraDiff['gildedCrest'] + extraDiff['safetyComponent'] + extraDiff['missive']
all_data['difficulty_gilded_missive_embellishment'] = all_data['difficulty'] + extraDiff['gildedCrest'] + extraDiff['missive'] + extraDiff['embellishment']
all_data['difficulty_combatant'] = all_data['difficulty']+extraDiff['combatant']
all_data['difficulty_combatant_missive'] = all_data['difficulty']+extraDiff['combatant']+extraDiff['missive']
all_data['difficulty_combatant_embellishment'] = all_data['difficulty']+extraDiff['combatant']+extraDiff['embellishment']
all_data['difficulty_combatant_missive_embellishment'] = all_data['difficulty']+extraDiff['combatant']+extraDiff['missive']+extraDiff['embellishment']
all_data['difficulty_aspirant'] = all_data['difficulty']+extraDiff['aspirant']
all_data['difficulty_aspirant_missive'] = all_data['difficulty']+extraDiff['aspirant']+extraDiff['missive']
all_data['difficulty_aspirant_embellishment'] = all_data['difficulty']+extraDiff['aspirant']+extraDiff['embellishment']
all_data['difficulty_aspirant_missive_embellishment'] = all_data['difficulty']+extraDiff['aspirant']+extraDiff['missive']+extraDiff['embellishment']
all_data['difficulty_gladiator'] = all_data['difficulty']+extraDiff['gladiator']
all_data['difficulty_gladiator_missive'] = all_data['difficulty']+extraDiff['gladiator']+extraDiff['missive']
all_data['difficulty_gladiator_embellishment'] = all_data['difficulty']+extraDiff['gladiator']+extraDiff['embellishment']
all_data['difficulty_gladiator_missive_embellishment'] = all_data['difficulty']+extraDiff['gladiator']+extraDiff['missive']+extraDiff['embellishment']

In [ ]:
modifiers = ['', '_safetycomponent', '_missive', '_embellishment', '_safetycomponent_missive', 
             '_missive_embellishment', '_weathered', '_weathered_safetycomponent', 
             '_weathered_safetycomponent_missive', '_weathered_missive', '_weathered_embellishment',
             '_weathered_missive_embellishment', '_runed', '_runed_safetycomponent', '_runed_safetycomponent_missive', '_runed_missive',
             '_runed_embellishment', '_runed_missive_embellishment', '_gilded', '_gilded_safetycomponent', 
             '_gilded_missive', '_gilded_safetycomponent_missive', '_gilded_embellishment', 
             '_gilded_missive_embellishment', '_combatant', '_combatant_missive', '_combatant_embellishment', 
             '_combatant_missive_embellishment', '_aspirant', '_aspirant_missive', '_aspirant_embellishment', 
             '_aspirant_missive_embellishment', '_gladiator', '_gladiator_missive', '_gladiator_embellishment', 
             '_gladiator_missive_embellishment']
    
for modifier in modifiers:
    all_data['rank1mats_outcome'+modifier] = all_data.apply(lambda row: outcomeQuality(row['skill1'], 
                                                                                       row['difficulty'+modifier], 
                                                                                       row['tag']), axis=1)
    all_data['rank2mats_outcome'+modifier] = all_data.apply(lambda row: outcomeQuality(row['skill2'], 
                                                                                       row['difficulty'+modifier], 
                                                                                       row['tag']), axis=1)
    all_data['rank3mats_outcome'+modifier] = all_data.apply(lambda row: outcomeQuality(row['skill3'], 
                                                                                       row['difficulty'+modifier], 
                                                                                       row['tag']), axis=1)

In [ ]:
curTime = time.time()
times['difficulties'] = curTime - lastTime
lastTime = curTime

# Proper Case tag field

In [ ]:
all_data['tag'] = all_data['tag'].apply(lambda x: x.title() if x != "gear (pvp)" else "Gear (PvP)")

In [ ]:
curTime = time.time()
times['text formatting'] = curTime - lastTime
lastTime = curTime

# Remove pd.NA and Sorting

In [ ]:
all_data = all_data.reset_index(drop=True)

all_data['character'] = all_data['character'].replace({pd.NA:None, 'None':None})
all_data = all_data.sort_values(by=['profession', 'item'], ascending=[True, True])

assert(len(all_data['character'].unique()==8) or len(all_data['character'].unique()==9))

In [ ]:
curTime = time.time()
times['trimming and sorting'] = curTime - lastTime
lastTime = curTime

# File Saving

In [ ]:
items.to_pickle(STATIC_DIR+'items_TWW.pkl')

In [ ]:
all_data.to_pickle(STATIC_DIR+'data_TWW.pkl')

In [ ]:
curTime = time.time()
times['saving'] = curTime - lastTime
lastTime = curTime

In [ ]:
items = pd.read_pickle(STATIC_DIR+'items_TWW.pkl')
all_data = pd.read_pickle(STATIC_DIR+'data_TWW.pkl')

# Times

In [ ]:
curTime = time.time()
times['total'] = curTime - startTime

display(times)

# Issue Testing